In [1]:
%load_ext autoreload
%autoreload 2

from ble import get_ble_controller
from base_ble import LOG
from cmd_types import CMD
import time
import numpy as np

LOG.propagate = False

In [10]:
# Get ArtemisBLEController object
ble = get_ble_controller()

# Connect to the Artemis Device
ble.connect()

def handler(sender, msg):
    msg = msg.decode("utf-8")
    print(msg)

ble.start_notify(ble.uuid['RX_STRING'], handler)
ble.send_command(CMD.GET_TOF_DATA_5S, "")

2023-02-11 16:27:19,862 | INFO     |: Already connected to a BLE device
41544|28|61
41791|30|61
42044|30|61
42293|31|60
42538|30|60
42787|31|59
43040|31|59
43290|28|60
43539|30|59
43783|28|60
44032|28|62
44284|30|61
44530|29|60
44780|29|61
45032|27|60
45277|31|60
45526|28|63
45780|27|62
46027|27|60
46274|28|59
46522|28|60
46778|25|62
47020|29|60
47272|26|63
47522|23|61
2023-02-11 16:27:58,510 | INFO     |: Disconnected from C0:83:0C:66:2F:3C
